<a href="https://colab.research.google.com/github/jonkrohn/DLTFpT/blob/master/notebooks/transfer_learning_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning

In this notebook, we load a pre-trained model (in this case, VGGNet19) and finetune it for a new task: detecting hot dogs.

#### Load dependencies

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19 # new!
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator # new!

#### Load the pre-trained VGG19 model

In [ ]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

#### Freeze all the layers in the base VGGNet19 model

In [ ]:
for layer in vgg19.layers:
    layer.trainable = False

#### Add custom classification layers

In [ ]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

#### Compile the model for training

In [ ]:
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Prepare the data for training

In [ ]:
# You can comment out the two lines of code below if you executed them on
# a previous run of the notebook. The wget command downloads the data and the 
# tar command extracts the data from a compressed file format. 
! wget -c https://www.dropbox.com/s/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
! tar -xzf hot-dog-not-hot-dog.tar.gz

In [ ]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [ ]:
# Define the batch size:
batch_size=32

In [ ]:
# Define the train and validation generators: 
train_generator = train_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=15, 
                    epochs=16, validation_data=valid_generator, 
                    validation_steps=15)